[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

Initialize the environment and import the necessary library

In [2]:
!pip install uk-covid19

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [4]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [5]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
with open("timeseries.json", "rt") as input:
  jsondata=json.load(input)

## Wrangle the data

To wrangle the raw data to Dataframe.
The data involved are:
* Cases: The number of cases that are published on the date.

* Hospital: The number of hospital admission that are published on the date.

* Death: the number of deaths are published on the date.



In [6]:


def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datadict = rawdata["data"]
    # initial the columns
    date=[]
    case=[]
    host=[]
    death=[]
    for d in datadict:
      date.append(d["date"])
      case.append(d["cases"])
      host.append(d["hospital"])
      death.append(d["deaths"])

    # sort the data by datetime
    res = pd.DataFrame({"date":date,"cases":case,"hospital":host,"deaths":death}).sort_values(by="date")
    res = res.fillna(0)
    return res


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(jsondata) # df is the dataframe for plotting

## Download current data

The “refresh” button callback can do the following
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* force a redraw of the graph and give the feedback.


In [7]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    # setting up the filters and structure for api
    filters = ['areaType=nation','areaName=England']
    structure = {
    "date": "date",
    "cases": "newCasesByPublishDate",
    "hospital": "newAdmissions",
    "deaths": "newDeathsByDeathDate"}

    api=Cov19API(filters=filters, structure=structure)

    try:
        apinewdata=api.get_json()  
    except:
        apibutton.button_style='warning'
        apibutton.icon='unlink'
        apibutton.description = "Unavailable"
    return apinewdata # return data read from the API

In [8]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    try:
      refresh_graph()
      # after all is done, you can switch the icon on the button to a "check" sign
      # and optionally disable the button - it won't be needed again. You can use icons
      # "unlink" or "times" and change the button text to "Unavailable" in case the 
      # api call fails.
      apibutton.icon="check"
    # apibutton.disabled=True
    except:
      apibutton.icon="unlink"
      apibutton.description = "Unavailable"

    
apibutton=wdg.Button(
    description='Refresh', # you may want to change this...
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click for latest Public Health England data",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(button_style='info', description='Refresh', icon='exclamation-triangle', style=ButtonStyle(), tooltip='…

## Graphs and Analysis

In [9]:
# set up the widget
series=wdg.SelectMultiple(
    options=['cases', "hospital",'deaths'],
    value=['cases', "hospital",'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

# HBox 
controls=wdg.HBox([series, scale])

# timeserires graph for cases, hospital, and deaths
def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")


def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back
    
    
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)


Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*

Xinan Yang 220384201